In [21]:
from nltk.corpus import gutenberg

In [22]:
import nltk;
nltk.corpus.gutenberg.fileids()


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [23]:
a= nltk.corpus.gutenberg.fileids()

In [24]:
print(gutenberg.raw('austen-emma.txt')[0:1000])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died too long ago for her to have more than an indistinct
remembrance of her caresses; and her place had been supplied
by an excellent woman as governess, who had fallen little short
of a mother in affection.

Sixteen years had Miss Taylor been in Mr. Woodhouse's family,
less as a governess than a friend, very fond of both daughters,
but particularly of Emma.  Between _them_ it was more the intimacy
of sisters.  Even before Miss Taylor had ceased to hold the nominal
office of governess, the mildness o

In [25]:
from nltk.corpus import gutenberg   # Docs from project gutenberg.org
import re
files_en = gutenberg.fileids()      # Get file ids
doc_en = gutenberg.open('edgeworth-parents.txt').read()

tokens_en = re.sub('[^A-Za-z]', ' ', doc_en)
#nltk.download('gutenberg')
tokens_en_lower = tokens_en.lower()

from nltk.tokenize import word_tokenize
tokens_en_lower = word_tokenize(tokens_en_lower)
from nltk.corpus import stopwords 
filtered_words = [word for word in tokens_en_lower if word not in stopwords.words('english')]

en = nltk.Text(filtered_words)

print(len(en.tokens))       # returns number of tokens (document length)
print(len(set(en.tokens)))  # returns number of unique tokens
en.vocab() 

#en.plot(50)
print(len(doc_en))
print(tokens_en.count('Emma'))
print(en.count('Emma'))

78118
8169
935158
0
0


In [26]:
len(filtered_words)

78118

In [28]:
def sampling_6(data, n=10):
    data=copy.deepcopy(data);
    sample=[];
    for i in range(n):
        idx = random.randint(0,len(data)-1);
        a =sample.append(data.pop(idx));
    return sample

In [29]:
import random
import copy
b=[]
for i in range(200): 
    a=sampling_6(filtered_words, n=150)
    b.append(a)
    

In [30]:
import pandas as pd
df = pd.DataFrame(b)


In [32]:
df

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,conquered,try,would,striking,wales,asked,show,hardy,endeavoured,water,...,speak,well,wood,could,honour,silver,one,pins,mr,engage
1,old,toast,especially,go,cried,man,built,many,meaning,esteem,...,come,kept,upon,knowledge,susan,susan,say,may,last,worth
2,coins,plunged,extremely,said,mason,appeared,ear,month,talb,going,...,bending,presents,pounds,taught,pudding,convinced,minutes,silent,thing,principal
3,must,difficult,like,work,notwithstanding,well,mountain,fair,give,villa,...,entr,may,head,never,considerable,mrs,promises,night,louisa,somebody
4,tear,plentiful,people,entered,master,project,surface,could,every,francisco,...,charming,bring,track,resumed,let,heaven,scarcely,putting,time,fond
5,hurt,share,little,much,day,painted,somebody,let,wife,dinner,...,fisher,organ,going,sulphurous,day,old,little,make,troublesome,glad
6,expect,still,leaned,anger,daresay,know,sands,command,rest,like,...,let,used,mrs,mother,stop,confidence,house,loveit,mrs,father
7,indeed,stay,dark,spirits,thanks,half,said,dresses,carpenter,candles,...,even,play,perceived,thought,board,east,alone,see,stupidly,cried
8,told,told,wonder,height,therefore,pleasure,case,somers,let,pathetic,...,speech,found,fellow,paper,christopher,pretty,two,person,part,nice
9,never,let,proper,company,eye,purse,break,admired,words,hal,...,man,others,archer,said,something,domestic,father,fresh,opens,may


In [33]:
f=df.apply(" ".join, axis=1)

In [34]:
f

0      conquered try would striking wales asked show ...
1      old toast especially go cried man built many m...
2      coins plunged extremely said mason appeared ea...
3      must difficult like work notwithstanding well ...
4      tear plentiful people entered master project s...
5      hurt share little much day painted somebody le...
6      expect still leaned anger daresay know sands c...
7      indeed stay dark spirits thanks half said dres...
8      told told wonder height therefore pleasure cas...
9      never let proper company eye purse break admir...
10     leaning half halfpenny hours long without look...
11     said said breakfast heard way think generous u...
12     whoever bread fine little light afterwards mon...
13     enjoyed grave lounging merchants help changed ...
14     mind mrs length remained without hopes madam h...
15     penny filled example eight many surprised salt...
16     proved dark felt waited small soundly neighbou...
17     guinea agree lad history

In [55]:
f[0]

'fast determined next see shall wish running little rapture father tune uniform delicate first stay looked protect ever said door came young butcher dead faced paper sugar lengthened oh yards foot every de immediately parcel lost greco sometimes newington plainly last grey shall right let beg mats bogberry going john see nothing winning hatful whether market window employer constantly susan pies rendered ear triumphant everybody end guard everyone chambermaid sarve come father improve col yard hard likely moment song forms stuff say indeed ornament give therefore tell said ask sir means patients boy louisa large corner replied entrance blessing bit gardener wanted horse procured shall eat cautiously know bursal mad look hypocrite whole old sick poor franklin several honest foot better marianne food boy melancholy man employment known supper disorder greater glass frederick spin come best never boy trimmed accident back piece ran find litigious married figure thought hat flashed'

In [30]:
len(f[0])

991

In [31]:
len(f[15])

926

In [37]:
f[1]

'wonder whether summer ever know laughed slipped threw mr said show cried paradise poor ridgeway desirous dialogue quarter observed pshaw dressed bill arm quiet tight nephews refuges without money lightfoot said sure almonds advantage hear shoe archer saw perhaps made peddler help night said usually hold things brother passions head come would abbey dead also heard always matter pardon piedro seems either never went shilling sir afraid last neighbour things thanks message put caught friends quite nets substantially ready make good shillings jingle used ignorance house travelling head came species said mind unlock servants way dexterously else bridewell know love eggs talb twopence padrona means displeased wood whilst meant ha help lady saw maurice take third see china well upper provisions would mistake rose loveit fancy bracelet loveit mr miss gardener yet see guinea thinks look give loveit recollecting wish yesterday arrival completely wrong far cried former piedro see father'

In [36]:
for i in range(15):
    print(f[i])


fast determined next see shall wish running little rapture father tune uniform delicate first stay looked protect ever said door came young butcher dead faced paper sugar lengthened oh yards foot every de immediately parcel lost greco sometimes newington plainly last grey shall right let beg mats bogberry going john see nothing winning hatful whether market window employer constantly susan pies rendered ear triumphant everybody end guard everyone chambermaid sarve come father improve col yard hard likely moment song forms stuff say indeed ornament give therefore tell said ask sir means patients boy louisa large corner replied entrance blessing bit gardener wanted horse procured shall eat cautiously know bursal mad look hypocrite whole old sick poor franklin several honest foot better marianne food boy melancholy man employment known supper disorder greater glass frederick spin come best never boy trimmed accident back piece ran find litigious married figure thought hat flashed
wonder w

In [36]:
f = pd.DataFrame(f, columns=['Sample'])


In [37]:
f

,Sample
0,conquered try would striking wales asked show ...
1,old toast especially go cried man built many m...
2,coins plunged extremely said mason appeared ea...
3,must difficult like work notwithstanding well ...
4,tear plentiful people entered master project s...
5,hurt share little much day painted somebody le...
6,expect still leaned anger daresay know sands c...
7,indeed stay dark spirits thanks half said dres...
8,told told wonder height therefore pleasure cas...
9,never let proper company eye purse break admir...


In [38]:
c = pd.DataFrame({'author':[],'Genre':[]})

In [39]:
c

,author,Genre


In [40]:
f 

,Sample
0,conquered try would striking wales asked show ...
1,old toast especially go cried man built many m...
2,coins plunged extremely said mason appeared ea...
3,must difficult like work notwithstanding well ...
4,tear plentiful people entered master project s...
5,hurt share little much day painted somebody le...
6,expect still leaned anger daresay know sands c...
7,indeed stay dark spirits thanks half said dres...
8,told told wonder height therefore pleasure cas...
9,never let proper company eye purse break admir...


In [41]:
frames =[f,c]

In [42]:
result = pd.concat(frames)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [43]:
result = result[['Sample','author','Genre']]

In [193]:
for i in range(200):
    result['Genre'] ="Fiction"
    result['author'] =0

In [162]:
result

,Sample,author,Genre
0,conquered try would striking wales asked show ...,a,Fiction
1,old toast especially go cried man built many m...,a,Fiction
2,coins plunged extremely said mason appeared ea...,a,Fiction
3,must difficult like work notwithstanding well ...,a,Fiction
4,tear plentiful people entered master project s...,a,Fiction
5,hurt share little much day painted somebody le...,a,Fiction
6,expect still leaned anger daresay know sands c...,a,Fiction
7,indeed stay dark spirits thanks half said dres...,a,Fiction
8,told told wonder height therefore pleasure cas...,a,Fiction
9,never let proper company eye purse break admir...,a,Fiction


In [168]:
    from sklearn.feature_extraction.text import CountVectorizer
matrixBoW = CountVectorizer(max_features=1000)
XBoW = matrixBoW.fit_transform(result['Sample']).todense()
XBoW= pd.DataFrame(XBoW, columns= matrixBoW.get_feature_names())

In [189]:
from sklearn import preprocessing
XBoW['label']='a'
for column in XBoW.columns:
    if XBoW[column].dtype == type(object):
        le = preprocessing.LabelEncoder()
        XBoW[column] = le.fit_transform(XBoW[column])

In [190]:
XBoW

,abbey,able,absolutely,account,acquainted,across,added,admirable,affair,affected,...,write,wrong,ye,year,years,yes,yesterday,yet,young,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,2,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [173]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrixTfidf = TfidfVectorizer()
XTfidf = matrixTfidf.fit_transform(result['Sample']).todense()
XTfidf= pd.DataFrame(XTfidf, columns= matrixTfidf.get_feature_names())

In [174]:
XTfidf['Label']='a'
XTfidf

,abated,abbey,abilities,able,abroad,abruptly,absence,absences,absolute,absolutely,...,yet,yield,yielded,yonder,york,young,youngest,youth,zeal,Label
0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,a
1,0.0,0.079711,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,a
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,...,0.000000,0.0,0.116055,0.000000,0.0,0.000000,0.0,0.0,0.0,a
3,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,a
4,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,...,0.056129,0.0,0.000000,0.000000,0.0,0.057831,0.0,0.0,0.0,a
5,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,a
6,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,a
7,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,a
8,0.0,0.000000,0.0,0.073674,0.0,0.0,0.000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,a
9,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,a


In [197]:

y = result['author']
y[0]= 1256
y[32]= 6474
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XBoW, y)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [177]:
y = result['author']

# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XTfidf, y)


In [198]:
# Naive Bayes 
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predict Class
y_pred = classifier.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

In [201]:
y_test

110       0
115       0
135       0
9         0
108       0
56        0
161       0
152       0
32     6474
177       0
140       0
106       0
22        0
43        0
129       0
163       0
77        0
168       0
2         0
1         0
184       0
57        0
63        0
157       0
21        0
46        0
188       0
35        0
65        0
143       0
52        0
112       0
87        0
103       0
39        0
3         0
15        0
7         0
137       0
27        0
180       0
136       0
5         0
66        0
83        0
45        0
193       0
199       0
86        0
170       0
Name: author, dtype: int64

In [199]:
accuracy

0.98

In [108]:
classifier

GaussianNB(priors=None, var_smoothing=1e-09)

In [109]:
y_pred

array(['Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth',
       'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth',
       'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth',
       'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth',
       'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth',
       'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth',
       'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth',
       'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth',
       'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth',
       'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth', 'Edgeworth'],
      dtype='<U9')